In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import pandas as pd
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
import xgboost as xgb 
from sklearn.model_selection import GroupKFold
import time
import matplotlib.pyplot as plt

In [5]:
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'

In [6]:
test_sessions = pd.read_csv(test_sessions_path)

In [7]:
test_sessions

,prev_items,locale
0,['B087VLP2RT' 'B09BRQSHYH' 'B099KW4ZLV'],DE
1,['B08XW4W667' 'B096VMCJYF' 'B096VMCJYF'],DE
2,['B09Z4T2GJ3' 'B09Z3FBXMB' 'B0936K9LTJ' 'B09Z4...,DE
3,['B07T6Y2HG7' 'B07T2NBLX9' 'B07Y1G5F3Y'],DE
4,['B0B2DRKZ6X' 'B0B2DRKZ6X' 'B0B2DRKZ6X'],DE
...,...,...
316967,['B078RJX3CC' 'B07GKM97YF'],UK
316968,['B01LX5Y7RG' 'B00M35Y2J0' 'B0BFR9D1Y2' 'B09BB...,UK
316969,['B09HGRXXTM' 'B08VDNCZT9'],UK
316970,['B089CVQ2FS' 'B089CVQ2FS'],UK


In [3]:
submission = pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/predictions/XGBoost/xgboost_06_04_10_24_with_feat_models.parquet', engine='pyarrow')

In [8]:
submission

,next_item_prediction,locale
0,"[B091CK241X, B07SDFLVKD, B0BGC82WVW, B093X59B3...",DE
1,"[B084CB7GX9, B004P4QFJM, B08HQWQ1SK, B004P4OF1...",DE
2,"[B09Z4PZQBF, B08LLF9M11, B09GPJ15GS, B09KBCTXF...",DE
3,"[B07Y1KLF25, B07T5XY2CJ, B07T2NFRFV, B07T5XJW9...",DE
4,"[B0B2JY9THB, B08SXLWXH9, B01MRXVY2O, B08SHZHRQ...",DE
...,...,...
316967,"[B07GKP2LCF, B07GKYSHB4, B00V6FIFZ0, B09QMK82R...",UK
316968,"[B00M35Y326, B07KTJXWHH, B08L5Z8GPL, B00NVMIO0...",UK
316969,"[B08VDHH6QF, B08VD5DC5L, B09HGSCL9Q, B08VDSL59...",UK
316970,"[B089CZWB4C, B08W2JJZBM, B09WCQYGX8, B08T1ZJYH...",UK


In [11]:
for row in tqdm(test_sessions.itertuples(), total=test_sessions.shape[0]):
    idx = row.Index
    prev_items = eval(row.prev_items.replace(' ', ','))
    next_item_prediction = submission.iloc[idx]['next_item_prediction']
    next_item_prediction = np.insert(next_item_prediction, 50, prev_items[-1])
    if len(prev_items) >= 2:
        next_item_prediction = np.insert(next_item_prediction, 51, prev_items[-2])
    submission.iloc[idx]['next_item_prediction'] = next_item_prediction[:100]

100%|██████████| 316972/316972 [01:02<00:00, 5088.79it/s]


In [12]:
len(submission.iloc[0]['next_item_prediction'])

100

In [14]:
test_sessions.iloc[0]['prev_items']

"['B087VLP2RT' 'B09BRQSHYH' 'B099KW4ZLV']"

In [15]:
submission.iloc[0]['next_item_prediction'][50], submission.iloc[0]['next_item_prediction'][51]

('B099KW4ZLV', 'B09BRQSHYH')

In [18]:
test_sessions.iloc[15000]['prev_items']

"['B08H5JP1Q4' 'B00NUS53CY']"

In [19]:
submission.iloc[15000]['next_item_prediction'][50], submission.iloc[15000]['next_item_prediction'][51]

('B00NUS53CY', 'B08H5JP1Q4')

In [20]:
submission.to_parquet('../predictions/XGBoost/xgboost_06_04_15_03_add_userhist.parquet', engine='pyarrow')